## Import

In [11]:
from dotenv import load_dotenv
import os
import json
from h2ogpte import H2OGPTE
from flask import Flask, render_template, request, session
from flask_wtf import FlaskForm
from wtforms import FileField
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 

app = Flask(__name__)
app.secret_key = 'your_secret_key'  


## loading

In [13]:
@app.route('/loading', methods=['GET'])
def upload_file(data_dir):
    client = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key= H2O_API_KEY,
    )

    # Create a new collection
    collection_id = client.create_collection(
        name='devMeetingAssistant',
        description='Test development for meeting assistant (prompt engineering)',
    )
    with open(data_dir, 'rb') as f:
        meeting_data = client.upload('Data/fake_sample.txt', f)
    client.ingest_uploads(collection_id, [meeting_data,])
    session['client'], session['collection_id'], session['meeting_data'] = client, collection_id, meeting_data
    return render_template('loading.html',form=form)




In [16]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm='h2oai/h2ogpt-4096-llama2-70b-chat',
        )
        return answer
    
def extract_json_string(full_string):
    start_index = full_string.find('{')  # Find the index of the first '{'
    end_index = full_string.rfind('}')   # Find the index of the last '}'
    
    if start_index != -1 and end_index != -1:
        return full_string[start_index:end_index + 1]
    else:
        return None
    
def string_like_JSON_to_txt(reply):
    json_str = extract_json_string(reply.content)
    
    if json_str is None:
        return "Invalid JSON format. Cannot parse the response."

    # Parse the extracted JSON string
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        return "Error parsing JSON data."

    # Extract information from the parsed data
    agenda = data.get('Agenda', '')
    meeting_summary = data.get('Meeting Summary', '')
    actionables = data.get('Actionables', [])

    # Format the post-meeting email
    email_subject = f"Post-Meeting Summary: {agenda}"
    email_body = f"Dear Team,\n\nHere's a summary of our recent meeting:\n\nAgenda: {agenda}\n\nMeeting Summary:\n{meeting_summary}\n\nActionables:\n"
    for action in actionables:
        email_body += f"- {action['Action']} (Assigned to: {action['Assigned']}, Deadline: {action['Deadline']}, Priority: {action['Priority']})\n"

    email_body += "\nPlease let me know if anything needs clarification or if there are additional action items to add.\n\nBest regards,\n[Your Name]"
    return email_body

In [ ]:
def get_summary_response(client, chat_session_id):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    main_prompt = """
    Execute all the following steps: \
    1. Examine the document and identify all names of the attendees, remember these names for the next steps.
    2. Identify and extract the main purpose or agenda of the meeting
    3. Identify all actionables mentioned during the meeting
    4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
    5. Summarize the entire transcript
    6. Present all identified information in the following JSON format for easy parsing:
    {"Agenda": "", "Meeting Summary":"", "Actionables":[{"Action":"", "Deadline":"", "Assigned":""}, ...]}
    DO NOT include anything other than the JSON format output in your response.
    """
    return rag_chat(client, chat_session_id, main_prompt, system_prompt)

def get_sentiment_response(client, chat_session_id, meeting_summary_response):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    sentiment_prompt = f"""
    Execute all the following steps: \
    1. Examine the contexts of this JSON object: {meeting_summary_response}
    2. For each actionable, execute the following steps
    a. identify the sentence in the document where it is mentioned
    b. Interpret the sentence, based on this context, categorize the priority of this task as "High", "Medium" or "Low"
    c. Edit the JSON object, add a column to the dictionary for this action as follows: "Priority": ""
    3. Present all identified information in the following JSON format:
    {{"Agenda": "", "Meeting Summary":""}} for the entire meeting
    {{"Actionables":[{{"Action":"", "Deadline":"", "Assigned":"", "Priority":""}}]}} for each action plan
    """

    return rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)



In [ ]:
@app.route('/get_LLM_response', methods=['POST'])
def get_LLM_response(data_dir):
    client, collection_id = session.get('client'), session.get('collection_id')
    chat_session_id = client.create_chat_session(collection_id)
    # 1. Summary and action plan
    meeting_summary_response = get_summary_response(client, chat_session_id)

    # 2. Sentiment analysis/Priority allocation
    sentiment_reply = get_sentiment_response(client, chat_session_id, meeting_summary_response)

    # Convert to email body
    return string_like_JSON_to_txt(sentiment_reply)
    

## Prompt 3: Language Translation

In [ ]:
## havent thought about it yet lol

TODO:  
  
Output email .txt  
input user name for email sign-off?  
Encapsulate all functions  
Language Translation prompt  

In [ ]:
# with open("trial_meeting_minutes_output.txt","w") as f:
#     f.write(llm_reply.content)

# Emailing stuff that I havent looked at yet

## Convert output file into pdf file to attach to email

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf(text, filename='output.pdf'):
    # Create a canvas
    c = canvas.Canvas(filename, pagesize=letter)
    
    # Set up font
    c.setFont("Helvetica", 12)
    
    # Split text into lines
    lines = text.split('\n')
    
    # Set starting y-coordinate
    y = 750
    
    # Write text to the canvas
    for line in lines:
        c.drawString(50, y, line)
        y -= 15  # Adjust for next line
        
    # Save the canvas
    c.save()


In [ ]:
# Create PDF
create_pdf(llm_reply.content, "trial_meeting_minutes_output.pdf")

Automate Email sending

https://keentolearn.medium.com/automating-emails-with-python-a-comprehensive-guide-ba00fa98b92#:~:text=Once%20you%20have%20set%20up,account%2C%20and%20send%20the%20email.

## Created junk email for everyone to use
email : gpt10.4213.1@outlook.com <br>
pw: gpt10email<br>

name : gpt<br>
surname : 10<br>
country : senegal<br>
dob : 7 Mar 1989

#### Send email without attachment

In [ ]:
# #This works!
# import smtplib

# smtp_server = 'smtp-mail.outlook.com'
# smtp_port = 587
# smtp_username = 'gpt10.4213.1@outlook.com'
# smtp_password = 'gpt10email'

# from_email = 'gpt10.4213.1@outlook.com'
# to_email = 'gpt10.4213.1@outlook.com'
# subject = 'Hello, world!'
# body = 'This is a test email. using smtplib~~'

# message = f'Subject: {subject}\n\n{body}'

# with smtplib.SMTP(smtp_server, smtp_port) as smtp:
#     smtp.starttls()
#     smtp.login(smtp_username, smtp_password)
#     smtp.sendmail(from_email, to_email, message)

In [ ]:
smtp_server_mapping = {
    "gmail":"smtp.gmail.com",
    "outlook":"smtp-mail.outlook.com",
    "hotmail":"smtp.office365.com",
    "yahoo":"smtp.mail.yahoo.com",
}  #the rest dont care first since different smtp_port # ....

#### Send email with attachment

works with a list of email addresses ~ish

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

smtp_server = 'smtp-mail.outlook.com'
smtp_port = 587
smtp_username = 'gpt10.4213.1@outlook.com'
smtp_password = 'gpt10email'

from_email = 'gpt10.4213.1@outlook.com'
to_email = 'gpt10.4213.1@outlook.com'
to_email_list = ['gpt10.4213.1@outlook.com',]

meeting_date = "5/3/2024"
subject = f"{meeting_date} Meeting"  # meeting_date either user input or LLM search through document.
body = f'Please find attached the report. \n This is a summary of the meeting. \n {llm_reply2.content}'

msg = MIMEMultipart()
msg['From'] = from_email
# msg['To'] = to_email # send to single user
msg['To'] = ", ".join(to_email_list) # send to multiple users
msg['Subject'] = subject
msg.attach(MIMEText(body))

with open('trial_meeting_minutes_output.pdf', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='pdf')
    attachment.add_header('Content-Disposition', 'attachment', filename='report.pdf')
    msg.attach(attachment)

with open('trial_meeting_minutes_output.txt', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='txt')
    attachment.add_header('Content-Disposition', 'attachment', filename='report_friendly.txt')
    msg.attach(attachment)

with smtplib.SMTP(smtp_server, smtp_port) as smtp:
    smtp.starttls()
    smtp.login(smtp_username, smtp_password)
    smtp.send_message(msg)